In [37]:
import pandas as pd
CRITERIA_MMSE = ['MMDATE', 'MMYEAR', 'MMMONTH', 'MMDAY',
       'MMSEASON', 'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA', 'MMSTATE',
        'WORD1', 'WORD2', 'WORD3', 'MMD', 'MML', 'MMR',
       'MMO', 'MMW', 'WORD1DL', 'WORD2DL', 'WORD3DL',
       'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 'MMONFLR',
       'MMREAD', 'MMWRITE', 'MMDRAW']
CRITERIA_NPIQ = ['NPI'+chr(i)+'SEV' for i in range(65, 65+12)]
CRITERIA_MOCA = ["TRAILS","CUBE",
            "CLOCKCON","CLOCKNO","CLOCKHAN",
            "LION","RHINO","CAMEL",
            "IMMT1W1","IMMT1W2","IMMT1W3","IMMT1W4","IMMT1W5","IMMT2W1","IMMT2W2","IMMT2W3","IMMT2W4","IMMT2W5",
            "DIGFOR","DIGBACK",
            "LETTERS",
            "SERIAL1","SERIAL2","SERIAL3","SERIAL4","SERIAL5",
            "REPEAT1","REPEAT2",
            "FFLUENCY",
            "ABSTRAN","ABSMEAS",
            "DELW1","DELW2","DELW3","DELW4","DELW5",
            "DATE","MONTH","YEAR","DAY","PLACE","CITY"]

def load_and_preprocess(data_paths, criterias, diagnosis_path, balance=False):
    if type(data_paths) is not list:
        data_paths = [data_paths]
    if type(criterias) is not list:
        criterias = [criterias]
    datas=[pd.read_csv(path)[criteria+['VISDATE','PTID']].dropna() for path, criteria in zip(data_paths, criterias)]
    diagnosis = pd.read_csv(diagnosis_path)[['EXAMDATE', 'PTID', 'DIAGNOSIS']].rename(columns={'EXAMDATE': 'VISDATE'}).dropna()
    diagnosis['DIAGNOSIS'] = diagnosis['DIAGNOSIS'] - 1  # Adjusting diagnosis labels if needed
    diagnosis['VISDATE'] = pd.to_datetime(diagnosis['VISDATE'])
    for data in datas:
        data['VISDATE'] = pd.to_datetime(data['VISDATE'])
        diagnosis = pd.merge_asof(diagnosis.sort_values(['VISDATE', 'PTID']), data.sort_values(['VISDATE', 'PTID']), on='VISDATE', by='PTID', direction='nearest', tolerance=pd.Timedelta('365D')).dropna()
    if balance:
        diagnosis = diagnosis.groupby('DIAGNOSIS').apply(lambda x: x.sample(diagnosis['DIAGNOSIS'].value_counts().min())).reset_index(drop=True)
    X=diagnosis[[item for sublist in criterias for item in sublist]]
    y=diagnosis['DIAGNOSIS']
    return X,y
data_paths=['data/mmse.csv', 'data/npiq.csv', 'data/moca.csv']
criterias = [CRITERIA_MMSE, CRITERIA_NPIQ, CRITERIA_MOCA]
diagnosis_path = 'data/diagnosis.csv'
X, y = load_and_preprocess(data_paths,criterias,diagnosis_path, balance=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.82      0.78        11
         1.0       0.73      0.67      0.70        12
         2.0       0.88      0.88      0.88         8

    accuracy                           0.77        31
   macro avg       0.78      0.79      0.78        31
weighted avg       0.77      0.77      0.77        31



In [30]:
print(y.value_counts()) 
from sklearn.utils import resample



DIAGNOSIS
0.0    90
1.0    56
2.0    51
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
print(X.shape)
print(y.shape)

(0, 84)
(0,)


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.